In [110]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt

In [96]:
df = pd.read_csv("./data/transaction_df.csv", index_col=0,parse_dates=['transactionDateTime'])
print(df.columns)
# As proposed in Question_1_Load
df["Primary_key"] = df["transactionDateTime"].astype(str)+"_"+df["accountNumber"].astype(str)+"_"+df["cardLast4Digits"].astype(str)+"_"+df["cardPresent"].astype(str)+"_"+df["merchantName"].astype(str)+"_"+df["transactionAmount"].astype(str)+"_"+df["transactionType"].astype(str)
print("Total Count of proposed primary key",len(df["Primary_key"].unique()))
print("Total Count of df",len(df))

df["MultiSwipe_Transaction"] = 0

Index(['accountNumber', 'customerId', 'creditLimit', 'availableMoney',
       'transactionDateTime', 'transactionAmount', 'merchantName',
       'acqCountry', 'merchantCountryCode', 'posEntryMode', 'posConditionCode',
       'merchantCategoryCode', 'currentExpDate', 'accountOpenDate',
       'dateOfLastAddressChange', 'cardCVV', 'enteredCVV', 'cardLast4Digits',
       'transactionType', 'echoBuffer', 'currentBalance', 'merchantCity',
       'merchantState', 'merchantZip', 'cardPresent', 'posOnPremises',
       'recurringAuthInd', 'expirationDateKeyInMatch', 'isFraud'],
      dtype='object')
Total Count of proposed primary key 786363
Total Count of df 786363
2016-08-13 14:27:32_737265056_1803_False_Uber_98.55_PURCHASE


## Question 3: Data Wrangling - Duplicate Transactions

#### Can you programmatically identify reversed and multi-swipe transactions?

Yes

Reverse Transaction Conditions
Purchase is followed by a reversal
- Same Merchant
    - Same Account Number
        - Same transactionAmount
            - Transaction Type Reverse and Purchase

In [114]:
print(len(df[df["transactionType"]=="REVERSAL"]))

20303


In [115]:
count_reversal = 0
count_reversal_only = 0
count_reversal_amount = 0
for merchant in df["merchantName"].unique():
    df_merchant = df[df["merchantName"]==merchant]
    for accountnumber in df_merchant["accountNumber"].unique():
        df_accountnumber = df_merchant[df_merchant["accountNumber"]==accountnumber]        
        flag_purchase = 0
        flag_reversal = 0
        for transaction_amount in df_accountnumber["transactionAmount"].unique():
            df_transactiontype = df_accountnumber[df_accountnumber["transactionAmount"]==transaction_amount]
            for transaction_type in df_transactiontype["transactionType"].unique():
                    if transaction_type == "PURCHASE":
                        flag_purchase = 1
                    elif transaction_type == "REVERSAL":
                        flag_reversal = 1
            
        if flag_purchase==1 and flag_reversal ==1:
            count_reversal_amount +=transaction_amount
            count_reversal+=1
            
        if flag_reversal ==1:
            count_reversal_only+=1
                                        

In [120]:
print(count_reversal)
print(count_reversal_only)
print("Total Reversal Amount",count_reversal_amount)

14403
14508
Total Reversal Amount 2111586.0700000003


MultiSwipe Transaction Conditions:
- Purchase Transaction Type
    - Same Merchant 
        - Same Account Number
            - Same last 4 digit Card Number
                - Same transactionAmount
                    - TimeStamp similar

In [99]:
count = 0
total_sum = 0
multiswipe_transaction = {}
df_transactiontype = df[df["transactionType"]=="PURCHASE"] 
for merchant in df_transactiontype["merchantName"].unique():
    df_merchant = df_transactiontype[df_transactiontype["merchantName"]==merchant]

    for accountnumber in df_merchant["accountNumber"].unique():
        df_accountnumber = df_merchant[df_merchant["accountNumber"]==accountnumber]  
        
        for cardnumber in df_accountnumber["cardLast4Digits"].unique():
            df_cardnumber = df_accountnumber[df_accountnumber["cardLast4Digits"]==cardnumber]

            for transactionamt in df_cardnumber["transactionAmount"].unique():
                df_transactionamt = df_cardnumber[df_cardnumber["transactionAmount"]==transactionamt]
                df_transactionamt = df_transactionamt.sort_values(by='transactionDateTime', ascending=True)

                df_transactionamt.reset_index(inplace=True) # sort the transaction based on datetime so that we can choose the first transaction
                date_x = df_transactionamt["transactionDateTime"][0] #choose the first transaction as legit transaction
                for date_y in df_transactionamt["transactionDateTime"]: 
                    if (date_y-date_x).total_seconds()<1200 and (date_y-date_x).total_seconds()>0:  #All transaction with the same merchant name, transaction amount and time within 20min of initial transaction 
                            count+=1;
                            if accountnumber not in multiswipe_transaction.keys():
                                multiswipe_transaction[accountnumber] = 0

                            multiswipe_transaction[accountnumber]+=transactionamt
                            total_sum+=transactionamt
                    elif (date_y-date_x).total_seconds()>1200: #If the same customer goes twice to a shop not within the period of 20 min the new transaction should be taken as the legit and after that as multiswipe
                        date_x = date_y


In [68]:
print("Total Number of such multiswipe transaction",count)

Total Number of such multiswipe transaction 7450


Problem : We might lose out on transactions when customer forgets to purchase something during the first transaction and is purchasing again with the same amount as previous one - in this case it will be considered as multiswipe and not 2 legit transaction if they are both done withing 20min

In [64]:
print("Example of a valid Multiswipe")
df[df["accountNumber"]==863456981][df[df["accountNumber"]==863456981]["merchantName"]=='Lyft']  [df[df["accountNumber"]==863456981][df[df["accountNumber"]==863456981]["merchantName"]=='Lyft']  ["transactionAmount"]==136.37]

Example of a valid Multiswipe


,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,...,echoBuffer,currentBalance,merchantCity,merchantState,merchantZip,cardPresent,posOnPremises,recurringAuthInd,expirationDateKeyInMatch,isFraud
16868,863456981,863456981,10000,6512.50,2016-04-08 12:59:30,136.37,Lyft,US,US,9.0,...,NaN,3487.50,NaN,NaN,NaN,False,NaN,NaN,False,False
16869,863456981,863456981,10000,6376.13,2016-04-08 12:59:53,136.37,Lyft,US,US,9.0,...,NaN,3623.87,NaN,NaN,NaN,False,NaN,NaN,False,False
16870,863456981,863456981,10000,6239.76,2016-04-08 13:01:40,136.37,Lyft,US,US,9.0,...,NaN,3760.24,NaN,NaN,NaN,False,NaN,NaN,False,False


Further Investigation : I also wanted to add another column for Multiwise and Reversal transaction with bool type that identifies if its one of them

In [119]:
print("For every account")
n = 0
for key, values in multiswipe_transaction.items():
    n+=1;
    print("For account number = ",key, "Total Dollars in sum", values)
    if n==5:
        break

For every account
For account number =  114896048 Total Dollars in sum 919.19
For account number =  101380713 Total Dollars in sum 504.43000000000006
For account number =  205681025 Total Dollars in sum 341.75
For account number =  731428616 Total Dollars in sum 93.45
For account number =  863456981 Total Dollars in sum 1719.1099999999997


In [112]:
print("Total_amount",total_sum)

Total_amount 1103132.2500000033


What total number of transactions and total dollar amount do you estimate for the reversed transactions? For the multi-swipe transactions? (please consider the first transaction to be "normal" and exclude it from the number of transaction and dollar amount counts)

#### Reversal Transaction
Dollar amount = $2111586.0700000003

No Of such Reversal Transaction = 14403


#### MultiSwipe Transaction
Dollar amount = $1103132.2500000033

No Of such Reversal Transaction = 7450

Interesting About Such Transaction
1. Same Card and same account number at the same type can have multiple transaction at different merchants
2. Further Investigation : BY what time does the reversal transaction money get reflected
3. To change Multiswipe I can just add mutliple Reversal columns and add to the balance

### Questions

1. Reversal transactions and the above algorithm. why different - which case is not being satisfied - further investigate